# Automated Question-Answer Generation from Pharmaceutical Data

This notebook systematically generates question-answer pairs from pharmaceutical data sources including:
1. PubMed target passages
2. DrugBank tables
3. Related pharmaceutical data

Each QA pair will include:
- Question
- Answer
- Source text/passage
- Related table

In [27]:
# Import required libraries
import os
import pandas as pd
import json
from pathlib import Path
from openai import OpenAI
import csv
from tqdm import tqdm
from collections import defaultdict
import re

In [39]:
# Configuration and paths
PUBMED_TARGETS_DIR = '../data/Pharma/pubmed-targets'
DRUGBANK_TABLES_DIR = '../data/Pharma/drugbank-tables'
MAPPING_FILE = '../data/Pharma/pubmed-drugbank-tables.gt'
OUTPUT_FILE = 'table_output_2.gt'

# Initialize your LLM API key if needed
# os.environ["OPENAI_API_KEY"]="<KEY>"
client = OpenAI()


In [3]:
def load_passage_table_mapping():
    """Load the mapping between passages and their relevant tables"""
    mapping = defaultdict(list)
    with open(MAPPING_FILE, 'r') as f:
        for line in f:
            passage_id, table_name = line.strip().split(',')
            mapping[passage_id].append(table_name)
    # print("mapping", mapping)
    return mapping

def load_target_passages():
    """Load all target passages from the pubmed-targets directory"""
    passages = {}
    target_files = Path(PUBMED_TARGETS_DIR).glob('Target-*')
    
    for file_path in target_files:
        target_id = file_path.name
        with open(file_path, 'r') as f:
            passages[target_id] = f.read()
    # print("passages", passages)
    return passages

def load_drugbank_tables():
    """Load all relevant DrugBank tables"""
    tables = {}
    csv_files = Path(DRUGBANK_TABLES_DIR).glob('*.csv')
    
    for file_path in csv_files:
        table_name = file_path.stem
        tables[table_name] = pd.read_csv(file_path)
    
    # print("tables", tables)
    return tables

In [15]:
# def get_relevant_table_content(table_id, table_content, max_rows=5):
#     # """Extract relevant content from tables for context"""
#     # print("Debug - Available tables:", tables.keys())
#     # print("Debug - Looking for table_names:", table_names)

#     total_max_rows = 500
#     max_rows_per_table = 500
#     table_content = {}
#     remaining_rows = total_max_rows
#     valid_tables = [name.replace('.csv', '') for name in table_names if name.replace('.csv', '') in tables]
    
#     # Calculate rows per table based on available tables
#     if valid_tables:
#         rows_per_table = min(max_rows_per_table, remaining_rows // len(valid_tables))
#     else:
#         rows_per_table = 0

#     table_content = {}
#     for table_name in table_names:
#         # Remove .csv extension if present
#         base_table_name = table_name.replace('.csv', '')
        
#         if base_table_name in tables:
#             df = tables[base_table_name]
            
#             # Take fewer rows for large tables
#             rows_to_take = min(rows_per_table, len(df))
            
#             table_content[base_table_name] = {
#                 'columns': list(df.columns),
#                 'data': df.head(rows_to_take).to_dict('records'),
#                 'total_rows': len(df)  # Include total row count for reference
#             }
            
#             remaining_rows -= rows_to_take
#         else:
#             print(f"Debug - Table '{base_table_name}' not found in available tables")
#     return table_content

def generate_questions_for_table(table_ids, table_contents, model="gpt-4o"):
    """Generate questions for a given passage and its relevant tables using LLM"""

    # Total token budget for all tables combined
    max_tokens = 10000
    
    # Allocate tokens equally among all tables
    tokens_per_table = max_tokens // len(table_ids) if table_ids else 0
    
    formatted_tables = []
    
    # Process each table
    for i, (table_id, table_content) in enumerate(zip(table_ids, table_contents)):
        df = table_content
        
        # Function to estimate tokens in a row (rough approximation: ~4 chars per token)
        def estimate_row_tokens(row):
            row_str = str(row.to_dict())
            return len(row_str) // 4
        
        # Sample rows to estimate average row size
        sample_size = min(5, len(df))
        if sample_size > 0:
            sample_rows = df.sample(sample_size) if len(df) > 5 else df
            avg_tokens_per_row = sum(estimate_row_tokens(row) for _, row in sample_rows.iterrows()) // max(1, sample_size)
            
            # Calculate rows we can include based on allocated tokens
            rows_to_take = min(tokens_per_table // max(1, avg_tokens_per_row), len(df))
            
            # Take at least 3 rows but no more than 50 per table
            rows_to_take = max(3, min(50, rows_to_take))
        else:
            rows_to_take = 0
        
        # Format this table
        formatted_table = {
            'table_name': table_id,
            'columns': list(df.columns),
            'data': df.head(rows_to_take).to_dict('records'),
            'total_rows': len(df)
        }
        
        formatted_tables.append(formatted_table)
    
    prompt = f"""
    Given the following passage and related tables, generate 2 meaningful question-answer pairs.
    IMPORTANT: Each question MUST be answerable using information from ONLY the tables. Each question MUST require synthesizing information from ALL tables to answer correctly.
    Only generate questions that require information from the tables. Try and make the question as difficult and technical as possible.
    
    You are a pharmaceutical educator creating realistic questions that a medical student, pharmacist, or healthcare professional might ask. Focus on clinically relevant information such as:
    - Drug mechanisms, indications, and therapeutic uses
    - Dosing considerations and administration routes
    - Side effects and contraindications
    - Drug interactions and pharmacological properties
    
    AVOID generating questions about:
    - Database-specific information (primary keys, IDs, URLs)
    - Technical metadata that wouldn't be useful in clinical practice
    - Questions that simply ask to list information without clinical context
    
    Examples of BAD questions to AVOID:
    - "What drug is linked to the FDA label with the URL ending in '1265922800,' and what is its state?"
    - "What is the primary key for the drug Denileukin diftitox, and what is its marketed mixture name?"
    
    Tables:
    {json.dumps(formatted_tables, indent=2)}
    
    Generate questions in the following format:
    1. question: [specific question about drug/treatment that would be asked in clinical practice]
       answer: [detailed answer combining information from the tables]
       text: None
       table: [table name(s) if information from table was used (e.g. drugbank-drug, drugbank-drug-interaction)]
    
    Ensure every question is clinically relevant and would be asked by real healthcare professionals.
    """
    
    # Call your LLM here with the prompt
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a medical and pharmaceutical expert tasked with generating detailed question-answer pairs about drugs and treatments."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    print(response.choices[0].message.content)
    
    # Parse the response into structured QA pairs
    qa_pairs = parse_llm_response(response.choices[0].message.content, table_ids)
    return qa_pairs

def parse_llm_response(response_text, table_ids):
    """Parse the LLM response into structured QA pairs"""
    qa_pairs = []
    
    # Split the response into individual QA entries
    entries = response_text.strip().split('\n\n')
    
    for entry in entries:
        if not entry.strip():
            continue
            
        lines = entry.strip().split('\n')
        current_qa = {
            'question': '',
            'answer': '',
            'text': 'None',
            'table': table_ids  # Default value
        }
        
        for line in lines:
            line = line.strip()
            # Skip empty lines and numbering
            if not line or line.replace('.', '').strip().isdigit():
                continue
                
            # Parse each field using more robust splitting
            if 'question:' in line:
                current_qa['question'] = line.split('question:', 1)[1].strip()
            elif 'answer:' in line:
                current_qa['answer'] = line.split('answer:', 1)[1].strip()
            elif 'text:' in line:
                current_qa['text'] = line.split('text:', 1)[1].strip()
            elif 'table:' in line:
                table_value = line.split('table:', 1)[1].strip()
                # Handle NA, N/A, None cases
                current_qa['table'] = 'None' if table_value.upper() in ['NA', 'N/A', 'NONE'] else table_value
        
        # Only add complete QA pairs that have both question and answer
        if current_qa['question'] and current_qa['answer']:
            qa_pairs.append(current_qa.copy())  # Use copy to avoid reference issues
    
    return qa_pairs

In [16]:
def verify_qa_uses_sources(qa_pair, content, model="gpt-4o"):
    # Create prompt for the LLM
    prompt = f"""
    Your task is to determine if the provided answer to a question uses information from the tables.
    
    Question: {qa_pair['question']}
    Answer: {qa_pair['answer']}
    
    Table Content:
    {content}
    
    Does the answer draw specific information from the table? Reply with just "YES" or "NO".
    If the answer only uses requires information ourtside of the content provided in the table, reply "NO".
    """
    
    # Call the LLM for verification
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a verification expert who determines if answers draw from multiple data sources."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1
    )
    
    verification = response.choices[0].message.content.strip().upper()
    return verification
    

In [45]:
def main():
    # # Load mappings and data
    # print("Loading passage-table mappings...")
    # passage_table_mapping = load_passage_table_mapping()
    
    # print("Loading target passages...")
    # passages = load_target_passages()
    
    print("Loading DrugBank tables...")
    tables = load_drugbank_tables()
    
    # Initialize output list
    qa_pairs = []
    
    # Process tables in pairs (two at a time)
    tables_list = list(tables.items())[:20]  # Limit to first 20 tables
    for i in range(0, len(tables_list), 2):
    # for i in range(0, len(tables_list)):
        # Get current pair (or single if at the end with odd number)
        current_tables = tables_list[i:i+2]
        
        # Extract ids and contents as separate lists
        table_ids = [table[0] for table in current_tables]
        print("table_ids", table_ids)
        table_contents = [table[1] for table in current_tables]
        
        # Generate QA pairs using two tables at a time
        new_qa_pairs = generate_questions_for_table(
            table_ids,
            table_contents
        )

        for qa_pair in new_qa_pairs:
            verification = verify_qa_uses_sources(qa_pair, table_contents)
            if verification == "YES":
                qa_pairs.append(qa_pair)
            else:
                print(f"QA pair {qa_pair['question']} does not use the table")  
        
    
    # Save results
    with open(OUTPUT_FILE, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(['question', 'answer', 'text', 'table'])  # header
        for qa_pair in qa_pairs:
            writer.writerow([
                qa_pair['question'],
                qa_pair['answer'],
                qa_pair['text'],
                qa_pair['table']
            ])
        
    print(f"Generated {len(qa_pairs)} question-answer pairs")

In [46]:
if __name__ == "__main__":
    main()

Loading DrugBank tables...
table_ids ['drugbank-targets_polypeptides_ext_id', 'drugbank-drug_prices']
1. question: What are the genetic target identifiers for the drug associated with Angiomax, and how might this influence its pharmacological action or therapeutic application?
   answer: The drug Angiomax is associated with the drug identifier "DB00006," which corresponds to the parent key "BE0000048" in the drugbank-targets_polypeptides_ext_id table. The genetic target identifiers for this drug include HGNC:3535 (HUGO Gene Nomenclature Committee), F2 (GenAtlas), M17262 (GenBank Gene Database), 339641 (GenBank Protein Database), 2362 (Guide to Pharmacology), P00734 (UniProtKB), and THRB_HUMAN (UniProt Accession). These identifiers suggest that Angiomax targets the gene associated with coagulation factor II (thrombin), which is critical in blood clotting processes. Understanding these genetic targets can aid in predicting the drug's pharmacological action and its application in conditio

In [47]:
def paraphrase_questions(input_csv, output_csv, model="gpt-4o"):
    """
    Paraphrase questions to remove references to specific tables or passages
    
    Args:
        input_csv: Path to input CSV file with original questions
        output_csv: Path to output CSV file for paraphrased questions
        model: LLM model to use for paraphrasing
    """
    # Read the input CSV
    df = pd.read_csv(input_csv, quoting=csv.QUOTE_ALL)
    
    # New list to hold the updated rows
    paraphrased_data = []
    
    # Process each question
    for _, row in tqdm(list(df.iterrows()), desc="Paraphrasing questions"):
        question = row['question']
        
        # Create prompt for the LLM
        prompt = f"""
        Your task is to paraphrase the following question to remove references to specific tables, datasets, or passages.
        If the question is solely about dataset structure or tables (e.g., "Does the dataset provide any links..."), return "REMOVE" to indicate it should be filtered out.
        Otherwise, paraphrase the question to make it more general while preserving its core content.

        Examples:
        - "Does the data set provide any links or references for Flumazenil in the drugbank-drugs_links table?" → "REMOVE"
        - "Which muscarinic receptor antagonist examined in the passage has affinity values closest to that of atropine and is also a biotech drug listed in the drugbank-drug table?" → "Which muscarinic receptor antagonist has affinity values closest to that of atropine and is also a biotech drug?"
        - "What is the significance of identifying Lys 190 as the modification site of HSA in the study, and how does it contrast with any protein-related drugs in the tables?" → "What is the significance of identifying Lys 190 as the modification site of HSA, and how does it contrast with any protein-related drugs?"

        Original question: {question}
        Paraphrased question:
        """
        
        # Call the LLM for paraphrasing
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an expert in reformulating pharmaceutical questions to make them more general and independent of specific data sources."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        
        paraphrased = response.choices[0].message.content.strip()
        
        # Skip questions that should be removed
        if paraphrased == "REMOVE":
            continue
            
        # Add the paraphrased question and original answer/text/table to the new data
        paraphrased_data.append({
            'question': paraphrased,
            'answer': row['answer'],
            'text': row['text'],
            'table': row['table']
        })
    
    # Write the paraphrased data to the output CSV
    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(['question', 'answer', 'text', 'table'])  # header
        for item in paraphrased_data:
            writer.writerow([
                item['question'],
                item['answer'],
                item['text'],
                item['table']
            ])
    
    print(f"Processed {len(df)} questions, kept {len(paraphrased_data)} after paraphrasing and filtering")
    return paraphrased_data

In [48]:
paraphrased_data = paraphrase_questions('table_output_2.gt', 'table_output_2_paraphrased.gt')

Paraphrasing questions: 100%|██████████| 9/9 [00:06<00:00,  1.43it/s]

Processed 9 questions, kept 9 after paraphrasing and filtering


In [49]:

def load_ground_truth(gt_file: str):
    """Load ground truth data from CSV file."""
    gt_data = []
    with open(gt_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            gt_data.append(row)
    return gt_data

def generate_llm_answer(question: str, model: str = "gpt-4o"):
    """
    Generate an answer to a question using an LLM.
    
    Args:
        question: The question to answer
        model: Model to use for generation
        
    Returns:
        The generated answer
    """
    try:
        # Create a simple prompt with just the question
        prompt = f"Answer this question with detailed information: {question}"
        print(f"Prompt: {prompt}")
        
        # Call OpenAI API
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,  # Use low temperature for more consistent answers
            max_tokens=500
        )
        
        # Extract the response text
        return response.choices[0].message.content.strip()
            
    except Exception as e:
        print(f"Error generating answer: {e}")
        time.sleep(2)  # Sleep to handle rate limits
        return ""

def calculate_llm_correctness(hypothesis: str, reference: str, question: str, model: str = "gpt-4o"):
    """
    Use an LLM to evaluate the correctness of the hypothesis compared to the reference.
    
    Args:
        hypothesis: The system-generated answer
        reference: The ground truth answer
        question: The original question
        model: Model to use for evaluation
        
    Returns:
        A score between 0 and 1 representing correctness (1 = fully correct, 0 = incorrect)
    """
    try:
        # Create prompt for the LLM
        prompt = f"""
You are an expert evaluator assessing the correctness of an answer to a question.

Question: {question}

Ground Truth Answer: {reference}

System Answer: {hypothesis}

Evaluate how correct the System Answer is compared to the Ground Truth Answer. Be very critical in your evaluation/analysis.
Give a score from 0 to 1 where:
- 1.0 means the System Answer is fully correct and contains all the information from the Ground Truth
- 0.0 means the System Answer is completely incorrect
- Values between 0 and 1 indicate partial correctness

Output a single line with just the score as a decimal between 0 and 1.
"""

        # Call OpenAI API
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,  # Use low temperature for more consistent evaluations
            max_tokens=300
        )
        
        # Extract the response text
        response_text = response.choices[0].message.content.strip()
        
        # Extract the score from the response - find the last number between 0 and 1
        score_matches = re.findall(r'(?:^|\s)(0(?:\.\d+)?|1(?:\.0+)?)(?:$|\s)', response_text)
        if score_matches:
            score = float(score_matches[-1])  # Take the last match as the final score
            return min(max(score, 0.0), 1.0)  # Ensure score is between 0 and 1
        else:
            print(f"Could not extract a score from LLM response: {response_text}")
            return 0.0
            
    except Exception as e:
        print(f"Error in LLM evaluation: {e}")
        time.sleep(2)  # Sleep to handle rate limits
        return 0.0

def filter_qa_pairs(gt_data,  llm_model: str = "gpt-4o", threshold: float = 0.5):
    """
    Filter question-answer pairs based on LLM correctness score.
    
    Args:
        gt_data: List of dictionaries containing ground truth data
        llm_model: Model to use for generation and evaluation
        threshold: Correctness threshold above which pairs are removed
        
    Returns:
        Filtered list of dictionaries
    """
    filtered_data = []
    
    for qa_pair in tqdm(gt_data, desc="Filtering QA pairs"):
        question = qa_pair['question']
        reference_answer = qa_pair['answer']
        
        # Generate an answer using just the question
        llm_answer = generate_llm_answer(question)
        
        # If we couldn't get an answer, keep the pair
        if not llm_answer:
            filtered_data.append(qa_pair)
            continue
        
        # Evaluate the correctness
        correctness_score = calculate_llm_correctness(
            llm_answer, reference_answer, question
        )
        
        print(f"Question: {question}")
        print(f"LLM Answer: {llm_answer[:100]}...")
        print(f"Correctness Score: {correctness_score}")
        
        # Keep pairs that the LLM couldn't answer correctly
        if correctness_score <= threshold:
            filtered_data.append(qa_pair)
    
    return filtered_data

def save_filtered_data(filtered_data, output_file):
    """Save filtered data to a CSV file."""
    if not filtered_data:
        print("No data to save!")
        return
    
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=filtered_data[0].keys(), quoting=csv.QUOTE_ALL)
        writer.writeheader()
        for row in filtered_data:
            writer.writerow(row)
    


In [50]:
# Load ground truth data
print(f"Loading ground truth data")
paraphrased_output_file = OUTPUT_FILE.replace(".gt", "_paraphrased.gt")
gt_data = load_ground_truth(paraphrased_output_file)
print(f"Loaded {len(gt_data)} QA pairs")

# Filter pairs
filtered_data = filter_qa_pairs(gt_data, 0.5)
print(f"Filtered to {len(filtered_data)} QA pairs")

filtered_output_file = OUTPUT_FILE.replace(".gt", "_correctness_filtered.gt")
# Save filtered data
save_filtered_data(filtered_data, filtered_output_file)
print(f"Saved filtered data to {filtered_output_file}")

# generate_llm_answer("What is the main target of the drug?")

Loading ground truth data
Loaded 9 QA pairs


Filtering QA pairs:   0%|          | 0/9 [00:00<?, ?it/s]

Prompt: Answer this question with detailed information: What are the genetic target identifiers for the drug associated with Angiomax, and how might these influence its pharmacological action or therapeutic application?


Filtering QA pairs:  11%|█         | 1/9 [00:08<01:06,  8.33s/it]

Question: What are the genetic target identifiers for the drug associated with Angiomax, and how might these influence its pharmacological action or therapeutic application?
LLM Answer: Angiomax is the brand name for bivalirudin, an anticoagulant medication primarily used during percut...
Correctness Score: 0.5
Prompt: Answer this question with detailed information: How does the pricing of Enbrel compare to that of Angiomax, and what might this suggest about their respective clinical applications?


...

Question: What impact might the mechanism of action of Multidrug resistance protein 1 (ABCB1) have on the pharmacokinetics of a drug like Prazosin?
LLM Answer: Multidrug resistance protein 1 (ABCB1), also known as P-glycoprotein (P-gp), is an important efflux ...
Correctness Score: 0.8
Prompt: Answer this question with detailed information: Considering their molecular weight and general function, which protein is more suitable for binding a greater variety of substances in the serum, Thyroxine-binding globulin or Serum albumin?


Filtering QA pairs:  89%|████████▉ | 8/9 [01:23<00:09,  9.94s/it]

Question: Considering their molecular weight and general function, which protein is more suitable for binding a greater variety of substances in the serum, Thyroxine-binding globulin or Serum albumin?
LLM Answer: When comparing Thyroxine-binding globulin (TBG) and Serum albumin in terms of their suitability for ...
Correctness Score: 0.9
Prompt: Answer this question with detailed information: Which enzyme is targeted by a protein that also interacts with the transporter known as MDR1_HUMAN, and what is the significance of this interaction in drug pharmacokinetics?


Filtering QA pairs: 100%|██████████| 9/9 [01:35<00:00, 10.66s/it]

Question: Which enzyme is targeted by a protein that also interacts with the transporter known as MDR1_HUMAN, and what is the significance of this interaction in drug pharmacokinetics?
LLM Answer: The enzyme targeted by a protein that also interacts with the transporter known as MDR1_HUMAN (also ...
Correctness Score: 0.0
Filtered to 4 QA pairs
Saved filtered data to table_output_2_correctness_filtered.gt
